# Build simple AML pipeline end-to-end

### Dataset

https://s3.amazonaws.com/datarobot_public_datasets/DR_Demo_Fire_Ins_Loss_only.csv

It is a regression problem, target `loss`.

### Task

Build a programm which

### 1. Reads provided data.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.options.display.max_columns = 100

In [2]:
url = 'https://s3.amazonaws.com/datarobot_public_datasets/DR_Demo_Fire_Ins_Loss_only.csv'
df = pd.read_csv(url)
df.head(2)

,loss,Exposure,Rating_Class,Sub_Rating_Class,Renewal_class,Sub_Renewal_Class,Property_size,Residents,Commercial,Norm_fire_risk,Norm_monthly_rent,Loan_mortgage,No_claim_Years,Previous_claims,Norm_area_m,Premium_remain,Premium_renew,Renewal_Type,crime_property_type,crime_residents,crime_area,crime_arson,crime_burglary,crime_neighbour_watch,crime_community,crime_risk,Geographical_risk,Weather_risk,ISO,ISO_cat,ISO_desc
0,0.07,3.0,H1,NaN,NaN,NaN,3.0,4.0,NaN,4.314082,758.287544,2.673639,2.251423,0.0,10.0,NaN,0.000000,B,1.209807,1.064398,1.066522,0.939346,1.708534,0.808015,1.040819,1.171366,0.284447,0.313157,5,ISO 5 - Modified or Semi Fire Resistive (MFR o...,which and floors. greater roof _—– modified an...
1,0.13,NaN,M1,NaN,NaN,NaN,6.0,1.0,NaN,4.171214,1060.660172,2.709158,2.084180,NaN,NaN,NaN,86.666667,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.006305,0.889677,6,ISO 6 - Fire Resistive (FR),bearing are walls deck/cover precast and/or co...


### 2. Does exploratory data analysis.


For this step you need to distinguish between numeric/categorical/text/date features. You can keep this step simple and provide a dict with mapping like column name -> type. Ideally type detection is done automatically using different heurictics.

#### Data overview

In [3]:
df.describe(include='all')

,loss,Exposure,Rating_Class,Sub_Rating_Class,Renewal_class,Sub_Renewal_Class,Property_size,Residents,Commercial,Norm_fire_risk,Norm_monthly_rent,Loan_mortgage,No_claim_Years,Previous_claims,Norm_area_m,Premium_remain,Premium_renew,Renewal_Type,crime_property_type,crime_residents,crime_area,crime_arson,crime_burglary,crime_neighbour_watch,crime_community,crime_risk,Geographical_risk,Weather_risk,ISO,ISO_cat,ISO_desc
count,1217.000000,210.000000,1130,5,124,103,1216.000000,1212.000000,998,1217.000000,1217.000000,196.000000,1217.000000,353.000000,1047.000000,184.000000,1217.000000,1217,923.000000,910.000000,923.000000,916.000000,922.000000,923.000000,903.000000,923.000000,1211.000000,1213.000000,1217.000000,1217,1217
unique,NaN,NaN,38,3,6,3,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,1217
top,NaN,NaN,H1,D,B,B,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ISO 1 Frame (combustible walls or roof),"materials, combustible or comments where (buil..."
freq,NaN,NaN,224,2,92,68,NaN,NaN,854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,287,1
mean,0.080477,2.809524,NaN,NaN,NaN,NaN,3.820724,2.514026,NaN,4.040762,1852.694145,3.286935,1.010640,0.320084,34.265521,69.201087,71.641081,NaN,1.002025,0.958211,0.989317,0.954384,1.044832,0.982719,0.962596,0.992618,1.125001,0.918192,2.889071,NaN,NaN
std,0.050425,0.726731,NaN,NaN,NaN,NaN,1.598589,1.318972,NaN,0.232445,1678.089181,0.243295,0.769526,0.500960,25.261731,43.405041,32.708544,NaN,0.439617,0.403047,0.407156,0.238484,0.523383,0.490376,0.254882,0.358018,1.918566,1.094440,1.598731,NaN,NaN
min,0.010000,1.000000,NaN,NaN,NaN,NaN,1.000000,1.000000,NaN,3.776480,500.000000,2.673639,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.083394,0.360578,0.065520,0.472193,0.154595,0.000000,0.522494,0.175763,0.000000,0.000000,1.000000,NaN,NaN
25%,0.040000,2.000000,NaN,NaN,NaN,NaN,3.000000,1.000000,NaN,3.898949,783.693499,3.144307,0.000000,0.000000,16.000000,4.000000,58.695652,NaN,0.699736,0.625186,0.687961,0.739102,0.572656,0.686813,0.739831,0.767780,0.091752,0.273559,2.000000,NaN,NaN
50%,0.070000,3.000000,NaN,NaN,NaN,NaN,3.500000,2.000000,NaN,3.898949,1232.882801,3.339025,1.048147,0.000000,30.000000,99.000000,85.564503,NaN,0.912543,0.903848,0.924562,0.942690,1.019386,0.989819,0.924495,0.974872,0.392954,0.550059,3.000000,NaN,NaN
75%,0.100000,3.000000,NaN,NaN,NaN,NaN,5.000000,4.000000,NaN,4.171214,2248.569323,3.474623,1.665109,0.832555,47.000000,100.000000,97.560858,NaN,1.209807,1.210431,1.190282,1.130489,1.322650,1.191823,1.110430,1.193020,1.059832,1.204226,4.000000,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1217 entries, 0 to 1216
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   loss                   1217 non-null   float64
 1   Exposure               210 non-null    float64
 2   Rating_Class           1130 non-null   object 
 3   Sub_Rating_Class       5 non-null      object 
 4   Renewal_class          124 non-null    object 
 5   Sub_Renewal_Class      103 non-null    object 
 6   Property_size          1216 non-null   float64
 7   Residents              1212 non-null   float64
 8   Commercial             998 non-null    object 
 9   Norm_fire_risk         1217 non-null   float64
 10  Norm_monthly_rent      1217 non-null   float64
 11  Loan_mortgage          196 non-null    float64
 12  No_claim_Years         1217 non-null   float64
 13  Previous_claims        353 non-null    float64
 14  Norm_area_m            1047 non-null   float64
 15  Prem

#### Determine the type of feature

In [5]:
def get_features_types(X, max_unique_for_categoric = 40):
    features_dict = {
        "numeric": [],
        "categoric": [],
        "text": [],
        "boolean": [],
        "constant": []
    }
    for col in X.columns:
        values_in_column = X[col]
        dtype = str(values_in_column.dtype)
        count_unique = values_in_column.fillna("missing").unique().shape[0]
        if count_unique == 1:
            features_dict["constant"].append(col)

        elif count_unique == 2:
            features_dict["boolean"].append(col)

        elif dtype == 'float64':
            features_dict["numeric"].append(col)

        elif dtype == 'int64':
            if count_unique <= max_unique_for_categoric:
                features_dict["categoric"].append(col)

            else:
                features_dict["numeric"].append(col)

        elif dtype == 'object':
            if count_unique <= max_unique_for_categoric:
                features_dict["categoric"].append(col)

            else:
                features_dict["text"].append(col)
                
    return features_dict

In [6]:
X = df.drop("loss", axis=1)
y = df["loss"]

features_dict = get_features_types(X, max_unique_for_categoric = 40)
#features_dict

In [7]:
from IPython.display import HTML

def display_features_types(features_dict):
    for key, values in features_dict.items():
        if len(values)>0:
            upper_key = key[0].upper()+key[1:]
            display(HTML("<h4>"+upper_key+" features</h4>"))
            display(values)

display_features_types(features_dict)

['Exposure',
 'Property_size',
 'Residents',
 'Norm_fire_risk',
 'Norm_monthly_rent',
 'Loan_mortgage',
 'No_claim_Years',
 'Previous_claims',
 'Norm_area_m',
 'Premium_remain',
 'Premium_renew',
 'crime_property_type',
 'crime_residents',
 'crime_area',
 'crime_arson',
 'crime_burglary',
 'crime_neighbour_watch',
 'crime_community',
 'crime_risk',
 'Geographical_risk',
 'Weather_risk']

['Rating_Class',
 'Sub_Rating_Class',
 'Renewal_class',
 'Sub_Renewal_Class',
 'Commercial',
 'ISO',
 'ISO_cat']

['ISO_desc']

['Renewal_Type']

### 3. (Optional) Does feature enginering.

You don't want to do things like one-hot encoding or normalization here cause it for different models it will de different. It is more about creating new features like extract day of moth/month of year from dates; or removing potentially useless features like IDs.

### 4. Does CV partitioning (keep 10-20% for holdout).

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0, shuffle=True)

### 5. Builds a pipeline which can train model.

As we discussed during the class, pipeline should be able to preprocess features (encoding, imputation, normalization), and fit a model.


```
categorical  ->  imputation        -> one-hot encoding ->
numeric      ->  imputation        -> standarization   -> linear regression
text         ->  tf-idf            ->                  ->
```

You can build multiple pipelines: for linear regression and tree based model like Random Forest. But single model pipeline also works. Note: lookout for preprocessing for linear models vs trees

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import ElasticNet
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import copy

In [10]:
numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=-99999)),
    ])

categoric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ('fx_selection', SelectFromModel(ElasticNet(alpha=0.7), threshold="1.25*mean"))
])


text_pipeline = Pipeline(steps=[
    ('bow', TfidfVectorizer(max_df=0.95, min_df=0.05, stop_words='english'))
])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, features_dict["numeric"]),
        ('cat', categoric_pipeline, np.concatenate((features_dict["categoric"],features_dict["boolean"]), axis=0)),
        ('text', text_pipeline, features_dict["text"][0])
    ])

# Append estimator to preprocessing pipeline.
# Now we have a full prediction pipeline.
est = Pipeline(steps=[('preprocessor', preprocessor),
                      ('rf', RandomForestRegressor(n_estimators=10, criterion='mae', n_jobs=-1))])

est.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('preprocessor', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('num', Pipeline(memory=None,
     steps=[('imputer', SimpleImputer(copy=True, fill_value=-99999, missing_values=nan,
       strategy='constant', v...imators=10, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False))])

In [11]:
print("model score: %.3f" % mean_absolute_error(y_test, est.predict(X_test)))

model score: 0.040


### 6. Optimizes hyperparameters of models. Use GridSearch, RandomSearch or Bayesian optimization.

#### GridSearch

In [12]:
%%time
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(estimator=est, param_grid={'rf__n_estimators':[10,20], 'rf__max_depth':[10,50,None]}, 
                  scoring='neg_mean_absolute_error', cv=3, refit=True, verbose=1, n_jobs=-1)

gs.fit(X_train, y_train)

print("best params: {}".format(gs.best_params_))
print("model score: %.3f" % mean_absolute_error(y_test, gs.best_estimator_.predict(X_test)))

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:   45.9s finished


best params: {'rf__max_depth': 50, 'rf__n_estimators': 20}
model score: 0.040
CPU times: user 25.7 s, sys: 82.3 ms, total: 25.8 s
Wall time: 53.1 s


#### RandomSearch

In [13]:
%%time

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint


param_dist = {
    'rf__n_estimators': randint(10,21), #The number of trees in the forest.
    'rf__max_depth': randint(10,51)  #The maximum depth of the tree.
    #'min_samples_split': randint(2,5),
    #'max_features': randint(2,11), #The number of features to consider when looking for the best split
    #'criterion': ['mse', 'mae']
}

rs = RandomizedSearchCV(estimator=est, param_distributions=param_dist, scoring='neg_mean_squared_error', n_iter=6, n_jobs=-1, cv=3, verbose=1)

rs = rs.fit(X,y)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:   45.8s finished


CPU times: user 24 s, sys: 33.2 ms, total: 24 s
Wall time: 52.5 s


In [14]:
print("best params: {}".format(rs.best_params_))
print("model score: %.3f" % mean_absolute_error(y_test, rs.best_estimator_.predict(X_test)))

best params: {'rf__max_depth': 35, 'rf__n_estimators': 16}
model score: 0.014


#### Bayesian optimization

In [32]:
%%time

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

opt = BayesSearchCV(
    est,
     {
        'rf__n_estimators':Integer(10, 21),
        'rf__max_depth':Integer(10, 51)
        #'rf__min_samples_split':Integer(2, 5),
        #'criterion': Categorical(['mse', 'mae'])
         
    },
    optimizer_kwargs={'base_estimator': 'RF'},
    n_iter=6,
    cv=4,
    scoring='neg_mean_squared_error',
    random_state=0
)
opt.fit(X_train, y_train)

print("val. score: %s" % opt.best_score_)
print("test score: %s" % opt.score(X_test, y_test))

val. score: -0.002065622521369547
test score: -0.002086468319246696
CPU times: user 4min 27s, sys: 952 ms, total: 4min 28s
Wall time: 1min 28s


In [33]:
opt.best_params_

OrderedDict([('rf__max_depth', 11), ('rf__n_estimators', 17)])

### 7. Reports scores of models.

In [28]:
def show_search_scores(opt, negative_loss = True):
    search_scores = pd.DataFrame(opt.cv_results_)
    search_scores = search_scores[["rank_test_score", "mean_test_score", "std_test_score", "param_rf__max_depth", "param_rf__n_estimators"]]
    search_scores = search_scores.sort_values("rank_test_score")
    if negative_loss:
        search_scores["mean_test_score"] = -1*search_scores["mean_test_score"]
    display(HTML(search_scores.to_html(index=False)))

#### Scores from GridSearch

In [29]:
show_search_scores(gs)

rank_test_score,mean_test_score,std_test_score,param_rf__max_depth,param_rf__n_estimators
1,0.036458,0.001900,50,20
2,0.036666,0.001678,10,20
3,0.037009,0.002330,None,10
4,0.037149,0.001741,None,20
5,0.037383,0.002410,10,10
6,0.037834,0.001779,50,10


#### Scores from RandomSearch

In [30]:
show_search_scores(rs)

rank_test_score,mean_test_score,std_test_score,param_rf__max_depth,param_rf__n_estimators
1,0.002504,0.001297,35,16
2,0.002586,0.001323,13,10
3,0.002608,0.001266,22,17
4,0.002611,0.001290,33,11
5,0.002631,0.001231,45,12
6,0.002706,0.001151,24,10


#### Scores from Bayesian optimization

In [34]:
show_search_scores(opt)

rank_test_score,mean_test_score,std_test_score,param_rf__max_depth,param_rf__n_estimators
1,0.002066,0.000270,11,17
2,0.002079,0.000276,24,15
3,0.002080,0.000226,27,17
4,0.002109,0.000285,39,13
5,0.002141,0.000221,14,15
6,0.002233,0.000258,40,10
